# O Problema do Transporte

## Definição do problema

O objetivo é minimizar o custo para transportar as mercadorias das fontes de origem para os diversos destinos (portos).

$\\$

<center>
**Conteineres Movimentados**
</center>

|        |Porto X|Porto Y|Porto Z|Soma   |
|:-------|-------|-------|-------|------:|
|Fonte A |       |       |       |105    |
|Fonte B |       |       |       |170    |
|Fonte C |       |       |       |130    |
|**Soma**|155    |120    |130    |405    |

$\\$
<center>
**Custos transportes**
</center>

|   |   X|   Y|   Z|
|:--|----|----|----|
| A | 7  | 12 | 14 |
| B | 12 | 10 | 13 |
| C | 7  | 9  | 11 |

### Função objetivo

Z = (
    7*x[0, 0] + 12*x[0, 1] + 14*x[0, 2] + 
    12*x[1, 0] + 10*x[1, 1] + 13*x[1, 2] + 
    7*x[2, 0] + 9*x[2, 1] + 11*x[2, 2]
)

### Variáveis de restrição

**capacidade:**

$
x[0, 0] + x[0, 1] + x[0, 2] = 105\\
x[1, 0] + x[1, 1] + x[1, 2] = 170\\
x[2, 0] + x[2, 1] + x[2, 2] = 130\\
$



**absorção:**

$
x[0, 0] + x[1, 0] + x[2, 0] = 155\\
x[0, 1] + x[1, 1] + x[2, 1] = 120\\
x[0, 2] + x[1, 2] + x[2, 2] = 130\\
$

In [79]:
import pandas as pd
import numpy as np
import io

In [82]:
def Z(x):
    return (
        7*x[0, 0] + 12*x[0, 1] + 14*x[0, 2] + 
        12*x[1, 0] + 10*x[1, 1] + 13*x[1, 2] + 
        7*x[2, 0] + 9*x[2, 1] + 11*x[2, 2]
    )

In [95]:
# cost matrix
values_text = '''
0,1,2
0,7,12,14
1,12,10,13
2,7,9,11
'''
df_cost = pd.read_csv(io.StringIO(values_text))

print('COSTS')
df_cost

COSTS


,0,1,2
0,7,12,14
1,12,10,13
2,7,9,11


In [ ]:
# restriction
capacity = [105, 170, 130]
absortion = [155, 120, 130]

## Northwest Corner

In [97]:
# initial basic solution with northwest corner
values_text = '''
0,1,2
0,0,0,0
1,0,0,0
2,0,0,0
'''
df_initial = pd.read_csv(io.StringIO(values_text))
df_initial

_capacity = list(capacity)
_absortion = list(absortion)


# iter by row
print(
    '#ID'.ljust(5, ' '), 
    'CAPACITY'.ljust(15, ' '), 
    'ABSORTION'.ljust(15, ' ')
)

ii = 1

print(
    ('#%s' % ii).ljust(5, ' '), 
    str(capacity).ljust(15, ' '), 
    str(absortion).ljust(15, ' ')
)

i = 0
while i < df_initial.shape[0]:
    # iter by column
    j = 0
    while j < df_initial.shape[1]:
        ii += 1
        r = _capacity[i] - _absortion[j]
        
        if r > 0:
            df_initial.iloc[i, j] = _absortion[j]
            _absortion[j] = 0
            _capacity[i] = r
        elif r < 0:
            df_initial.iloc[i+0, j] = _capacity[i]
            _capacity[i] = 0
            _absortion[j] = abs(r)
        else:
            df_initial.iloc[i+0, j] = _capacity[i]
            _capacity[i] = 0
            _absortion[j] = 0
            
        print(
            ('#%s' % ii).ljust(5, ' '), 
            str(_capacity).ljust(15, ' '), 
            str(_absortion).ljust(15, ' ')
        )
        
        if _capacity[i] == 0:
            break
        j += 1
    i += 1

df_initial

#ID   CAPACITY        ABSORTION      
#1    [105, 170, 130] [155, 120, 130]
#2    [0, 170, 130]   [50, 120, 130] 
#3    [0, 120, 130]   [0, 120, 130]  
#4    [0, 0, 130]     [0, 0, 130]    
#5    [0, 0, 130]     [0, 0, 130]    
#6    [0, 0, 130]     [0, 0, 130]    
#7    [0, 0, 0]       [0, 0, 0]      


,0,1,2
0,105,0,0
1,50,120,0
2,0,0,130


In [118]:
df = df_initial.copy()
# calculate the cost
z = Z(df.as_matrix())

# check the result
assert z == 105*7+ 50*12 + 120*10 + 130*11
print('Z =', z)

Z = 3965


In [130]:
def calc_n_used_cells(df: pd.DataFrame) -> int:
    """
    """
    cells = df.values.reshape(1, n*m)
    return cells[cells>0].shape[0]
    

def fix_table(df: pd.DataFrame) -> pd.DataFrame:
    """
    """
    n, m = df.shape
    n_used_cells = calc_n_used_cells(df)
    r = n_used_cells - (n+m-1)

    if r < 0:
        for i in range(n):
            for j in range(m):
                if df.iloc[i, j] == 0:
                    df.iloc[i, j] = -np.float('inf')
                    r += 1
                    if r >= 0:
                        break
            if r >= 0:
                break        
    return df

In [131]:
# check
print('\ninitial table')
df = df_initial.copy()
print(df)

df = fix_table(df)
print('\ntable fixed')
print(df)


initial table
     0    1    2
0  105    0    0
1   50  120    0
2    0    0  130

table fixed
     0           1    2
0  105        -inf    0
1   50  120.000000    0
2    0    0.000000  130


## Calculation of multipliers